In [ ]:
# using Voronoi diagrams and cosine similarity to find the top k vectors in a cluster
class VoronoiInvertedFileIndex:
    """
    A class representing an inverted file index based on Voronoi diagrams for nearest neighbor search.

    Attributes:
    - num_partitions (int): The number of Voronoi partitions or centroids.
    - centroids (numpy.ndarray): An array containing randomly generated centroids.
    - partition_indices (numpy.ndarray): An array storing the Voronoi partition indices for each data point.
    - partition_files (list): A list to store the paths of the partition files.

    Methods:
    - train(data): Train the Voronoi-based inverted file index with the given dataset.
    - query(query_vector, k): Perform a query to retrieve the top k nearest vectors to the given query vector.
    - generate_random_centroids(data): Generate random centroids within the data range.
    - assign_to_partitions(data): Assign data points to Voronoi partitions based on the closest centroid.
    - find_closest_partition(query_vector): Find the closest Voronoi partition to a query vector.
    - cosine_similarity(a, b): Compute the cosine similarity between vectors a and b.
    - save_partition_vectors(partition_index, vectors): Save partition vectors to a text file.
    """

    def __init__(self, num_partitions):
        """
        Initialize a VoronoiInvertedFileIndex instance.

        Parameters:
        - num_partitions (int): The number of Voronoi partitions or centroids.
        """
        self.num_partitions = num_partitions
        self.centroids = None
        self.partition_indices = None
        self.partition_files = None

    def train(self, data, output_folder='partition_files'):
        """
        Train the Voronoi-based inverted file index with the given dataset.

        Parameters:
        - data (numpy.ndarray): The dataset of vectors.
        - output_folder (str): The folder to store partition files.
        """
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        self.centroids = self.generate_random_centroids(data)
        self.partition_indices = self.assign_to_partitions(data)
        self.partition_files = [os.path.join(output_folder, f'partition_{i}.txt') for i in range(self.num_partitions)]

        for i in range(self.num_partitions):
            indices_in_partition = np.where(self.partition_indices == i)[0]
            partition_vectors = data[indices_in_partition]
            self.save_partition_vectors(i, partition_vectors)

    def query(self, query_vector, k):
        """
        Perform a query to retrieve the top k nearest vectors to the given query vector.

        Parameters:
        - query_vector (numpy.ndarray): The query vector.
        - k (int): The number of nearest vectors to retrieve.

        Returns:
        - result_indices (numpy.ndarray): The indices of the top k nearest vectors.
        """
        query_partition = self.find_closest_partition(query_vector)
        candidate_indices = np.where(self.partition_indices == query_partition)[0]

        # Assuming data is a global variable (modify as needed)
        similarities = np.dot(data[candidate_indices], query_vector.T)
        top_k_indices = np.argsort(similarities)[::-1][:k]

        return candidate_indices[top_k_indices]

    def generate_random_centroids(self, data):
        """
        Generate random centroids within the data range.

        Parameters:
        - data (numpy.ndarray): The dataset of vectors.

        Returns:
        - centroids (numpy.ndarray): An array containing randomly generated centroids.
        """
        return np.random.uniform(data.min(axis=0), data.max(axis=0), size=(self.num_partitions, data.shape[1]))

    def assign_to_partitions(self, data):
        """
        Assign data points to Voronoi partitions based on the closest centroid.

        Parameters:
        - data (numpy.ndarray): The dataset of vectors.

        Returns:
        - partition_indices (numpy.ndarray): An array storing the Voronoi partition indices for each data point.
        """
        return np.argmin(np.linalg.norm(data[:, np.newaxis, :] - self.centroids, axis=2), axis=1)

    def find_closest_partition(self, query_vector):
        """
        Find the closest Voronoi partition to a query vector.

        Parameters:
        - query_vector (numpy.ndarray): The query vector.

        Returns:
        - closest_partition (int): The index of the closest Voronoi partition.
        """
        return np.argmin(np.linalg.norm(self.centroids - query_vector, axis=1))

    @staticmethod
    def cosine_similarity(a, b):
        """
        Compute the cosine similarity between vectors a and b.

        Parameters:
        - a (numpy.ndarray): Vector a.
        - b (numpy.ndarray): Vector b.

        Returns:
        - similarity (float): Cosine similarity between vectors a and b.
        """
        return np.dot(a, b)

    def save_partition_vectors(self, partition_index, vectors):
        """
        Save partition vectors to a text file.

        Parameters:
        - partition_index (int): The index of the partition.
        - vectors (numpy.ndarray): The vectors in the partition.
        """
        np.savetxt(self.partition_files[partition_index], vectors)

# Example usage:
dimension = 70
num_vectors = 20000
data = np.random.random((num_vectors, dimension))

# Create and train the Voronoi-based inverted file index
voronoi_index = VoronoiInvertedFileIndex(num_partitions=5)
voronoi_index.train(data)

# Example query vector
query_vector = np.random.random((dimension,))

# Perform a query to get the top k nearest vectors
k = 5
result_indices = voronoi_index.query(query_vector, k)

print(f"Query Vector: {query_vector}")
print(f"Top {k} Nearest Vectors: {data[result_indices]}")
